In [1]:
# %load_ext autoreload

In [2]:
# %autoreload 2

In [3]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(1)

batch_size = 32
class_map = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}
show_summary = True
show_dataset_analyze = True

In [5]:
# Load Network and Print Summary
from DeepLib.model import CifarNetDilated
from DeepLib.utils import get_device, print_summary

# Check GPU availability
use_cuda, device = get_device()

In [6]:
from DeepLib.dataset import get_loader
from DeepLib.transform import get_a_train_transform, get_a_test_transform

train_loader, test_loader = get_loader('CIFAR10',get_a_train_transform(), get_a_test_transform(), batch_size=batch_size, use_cuda=use_cuda)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# # Calculate Class Scale
# from DeepLib.visualize import print_class_scale, print_samples
# if show_dataset_analyze:
#     # print_class_scale(train_loader, class_map)
#     print_samples(train_loader,class_map)# 

In [8]:
import time
import math

from DeepLib.backpropagation import train, test, get_sgd_optimizer
from DeepLib.utils import initialize_weights
from DeepLib.scheduler import one_cycle_lr_pt, one_cycle_lr_custom
from DeepLib.training import Training

# Train Params
epochs = 500
lr = 0.01
max_lr = 0.1
steps_per_epoch = len(train_loader)
dropout = False
momentum = 0.9
weight_decay = 0.000125

print("Using Device:", device)
print("Epochs:", epochs)
print("Lr:", lr)
print("Max Lr:", max_lr)
print("Batch Size:", batch_size)
print("Dropout:", dropout)
print("\n")

# bnmodel = CifarNet(norm='bn',base_channels=12).apply(initialize_weights).to(device)
model = CifarNetDilated().apply(initialize_weights).to(device)
print_summary(model, input_size=(3, 32, 32))


Using Device: cuda
Epochs: 500
Lr: 0.01
Max Lr: 0.1
Batch Size: 32
Dropout: False


Shape: torch.Size([2, 56, 8, 8])
Shape: torch.Size([2, 56, 1, 1])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 32, 32]             648
       BatchNorm2d-2           [-1, 24, 32, 32]              48
         Dropout2d-3           [-1, 24, 32, 32]               0
            Conv2d-4           [-1, 32, 30, 30]           6,912
       BatchNorm2d-5           [-1, 32, 30, 30]              64
         Dropout2d-6           [-1, 32, 30, 30]               0
            Conv2d-7           [-1, 32, 26, 26]           9,216
       BatchNorm2d-8           [-1, 32, 26, 26]              64
         Dropout2d-9           [-1, 32, 26, 26]               0
           Conv2d-10           [-1, 40, 26, 26]          11,520
      BatchNorm2d-11           [-1, 40, 26, 26]              80
        Dropout2d

In [9]:
optimizer = get_sgd_optimizer(model, lr=lr, weight_decay=weight_decay)

# Pytorch one cycle scheduler
pytorch_scheduler = one_cycle_lr_pt(
    optimizer, 
    lr=lr, 
    max_lr=max_lr, 
    steps_per_epoch=steps_per_epoch, 
    epochs=epochs, 
    anneal_strategy='linear'
)

# One cycle schedule with a custom function
custom_scheduler = one_cycle_lr_custom(
    optimizer, 
    lr=lr, 
    max_lr=max_lr, 
    steps_per_epoch=steps_per_epoch, 
    epochs=epochs, 
    anneal_strategy='linear'
)

trainer = Training(
    model,
    optimizer,
    custom_scheduler,
    train(),
    test,
    train_loader,
    test_loader,
    lr,
    epochs,
    device,
    dropout
)

trainer.run()

from DeepLib.visualize import plot_network_performance

# trainer.print_best_model()
plot_network_performance(epochs, trainer.schedule, trainer.list_train_loss, trainer.list_valid_loss, trainer.list_train_correct, trainer.list_valid_correct)

from DeepLib.utils import get_all_predictions, get_incorrrect_predictions, prepare_confusion_matrix
from DeepLib.visualize import plot_confusion_matrix

all_preds, all_targets = get_all_predictions(model, test_loader, device)
confusion_matrix = prepare_confusion_matrix(all_preds, all_targets, class_map)
plot_confusion_matrix(confusion_matrix, class_map, False)


from DeepLib.utils import get_incorrrect_predictions
from DeepLib.visualize import plot_incorrect_predictions

incorrect = get_incorrrect_predictions(model, test_loader, device)
plot_incorrect_predictions(incorrect, class_map)

| Epoch | LR       | Time    | TrainLoss | TrainCorrect | TrainAcc | ValLoss  | ValCorrect | ValAcc |
Shape: torch.Size([32, 56, 8, 8])
Shape: torch.Size([32, 56, 1, 1])
DataShape: torch.Size([32, 3, 32, 32])
Target Shape: torch.Size([32])
Output Shape: torch.Size([16, 10])


ValueError: Expected input batch_size (16) to match target batch_size (32).